In [ ]:
import numpy as np
import pandas as pd

data = pd.read_excel('./Detection_of_Soil_for_cotton_crop.xlsx', engine='openpyxl')
data

In [7]:
data.describe()


,Sample ID,ph,Temperature,Humidity,Density,Electrical Conductivity,N,P,K,Ca,Mg,Cotton Crop
count,48.00,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000
mean,24.50,6.819801,25.042049,66.221716,1.371875,6.716667,64.083333,55.083333,33.645833,18.672917,11.826042,0.333333
std,14.00,0.817940,4.463068,18.262961,0.191519,3.188573,43.625989,13.208368,17.982840,16.678087,5.200344,0.476393
min,1.00,4.371746,17.024985,15.658092,0.920000,0.670000,2.000000,18.000000,15.000000,3.820000,5.150000,0.000000
25%,12.75,6.337559,21.288970,60.326464,1.275000,4.967500,26.500000,47.000000,20.000000,6.117500,7.987500,0.000000
50%,24.50,6.859484,25.731311,71.584142,1.420000,6.430000,50.000000,56.000000,29.500000,8.985000,9.675000,0.000000
75%,36.25,7.348310,28.040214,78.028476,1.522500,9.085000,100.000000,60.250000,42.000000,31.797500,16.430000,1.000000
max,48.00,8.840656,36.891637,91.635362,1.680000,13.320000,143.000000,79.000000,85.000000,56.410000,24.540000,1.000000


In [8]:
cols = data.columns[data.dtypes==object]
cols

Index(['Grain Surface', 'Particle Attached', 'Particle Spacing',
       'Particle Width'],
      dtype='object')

In [9]:
for col in cols:
    print(col)
    print()
    print(data[col].value_counts())
    print('******************************')
    print()

Grain Surface

Grain Surface
scaly      16
gritty     15
smooth     11
fibrous     5
girtty      1
Name: count, dtype: int64
******************************

Particle Attached

Particle Attached
no    48
Name: count, dtype: int64
******************************

Particle Spacing

Particle Spacing
close      28
crowded    20
Name: count, dtype: int64
******************************

Particle Width

Particle Width
broad     36
narrow    11
Name: count, dtype: int64
******************************



In [10]:
# "Cotton Crop" is our label data and "Sample ID " is unwanted
target = data['Cotton Crop']
data.drop(['Sample ID','Cotton Crop'], axis=1, inplace=True)

In [11]:
target.value_counts()


Cotton Crop
0    32
1    16
Name: count, dtype: int64

In [12]:
df = data.drop(cols, axis=1)


In [13]:
from sklearn.preprocessing import MinMaxScaler
  
# copy the data
norm_df = pd.DataFrame(MinMaxScaler().fit_transform(df), columns=df.columns)

norm_df.head()

,ph,Temperature,Humidity,Density,Electrical Conductivity,N,P,K,Ca,Mg
0,0.476904,0.194032,0.873217,0.000000,0.532016,0.695035,0.524590,0.400000,0.497813,0.714286
1,0.596644,0.238866,0.851064,0.157895,0.053755,0.588652,0.655738,0.371429,0.160297,0.000000
2,0.776131,0.300980,0.745785,0.565789,0.453755,0.836879,0.606557,0.414286,0.555239,0.543063
3,0.583734,0.476482,0.848942,0.236842,0.076680,0.510638,0.278689,0.357143,0.154022,0.140794
4,0.728752,0.156302,0.736362,0.526316,0.488538,0.680851,0.393443,0.385714,0.767256,0.668901


In [15]:
! pip install xgboost


  Using cached xgboost-2.0.3-py3-none-win_amd64.whl.metadata (2.0 kB)
Using cached xgboost-2.0.3-py3-none-win_amd64.whl (99.8 MB)


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV,RepeatedStratifiedKFold

In [18]:
data['Particle Width']

0     narrow
1      broad
2      broad
3      broad
4      broad
5      broad
6      broad
7      broad
8      broad
9      broad
10     broad
11     broad
12     broad
13    narrow
14    narrow
15     broad
16     broad
17    narrow
18     broad
19     broad
20    narrow
21     broad
22     broad
23     broad
24     broad
25     broad
26     broad
27     broad
28     broad
29    narrow
30     broad
31    narrow
32     broad
33     broad
34     broad
35     broad
36       NaN
37     broad
38     broad
39    narrow
40     broad
41     broad
42     broad
43     broad
44    narrow
45    narrow
46    narrow
47     broad
Name: Particle Width, dtype: object

In [23]:
data['Particle Width'][data['Particle Width'].isna()]


36    NaN
Name: Particle Width, dtype: object

In [24]:
pw = data['Particle Width'].fillna(method='ffill')


C:\Users\lia68085\AppData\Local\Temp\ipykernel_3624\2803317121.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  pw = data['Particle Width'].fillna(method='ffill')


In [25]:
pw[36]

'broad'

In [26]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
pw = pd.DataFrame(enc.fit_transform(pw))
pw.value_counts()

0
0    37
1    11
Name: count, dtype: int64

In [28]:
ps = pd.DataFrame(enc.fit_transform(data['Particle Spacing']))
ps.value_counts()

0
0    28
1    20
Name: count, dtype: int64

In [29]:
target.value_counts()


Cotton Crop
0    32
1    16
Name: count, dtype: int64

In [30]:
est =[]
est.append(('LogisticRegression', Pipeline([('LogisticRegression', LogisticRegression())])))
est.append(('SVC', Pipeline([('SVC', SVC())])))
est.append(('GaussianNB', Pipeline([('GaussianNB', GaussianNB())])))
est.append(('BernoulliNB', Pipeline([('BernoulliNB', BernoulliNB())])))
est.append(('MultinomialNB', Pipeline([('MultinomialNB', MultinomialNB())])))
est.append(('KNeighborsClassifier', Pipeline([('KNeighborsClassifier', KNeighborsClassifier())])))
est.append(('DecisionTreeClassifier', Pipeline([('DecisionTreeClassifier', DecisionTreeClassifier())])))
# est.append(('XGBClassifier', Pipeline([('XGBClassifier', XGBClassifier())])))
est.append(('GradientBoostingClassifier', Pipeline([('GradientBoostingClassifier', GradientBoostingClassifier())])))
est.append(('AdaBoostClassifier', Pipeline([('AdaBoostClassifier', AdaBoostClassifier())])))
est.append(('RandomForestClassifier', Pipeline([('RandomForestClassifier', RandomForestClassifier())])))
est.append(('ExtraTreesClassifier', Pipeline([('ExtraTreesClassifier', ExtraTreesClassifier())])))
est.append(('BaggingClassifier', Pipeline([('BaggingClassifier', BaggingClassifier())])))

In [31]:
import warnings
warnings.filterwarnings(action='ignore')

seed = 4
splits = 5
repeats = 3

score = 'f1_weighted'
results_dict = {}
col_names  = ['Particle Spacing', 'Particle Width', 'Cotton Crop']
for index, column in enumerate([ps, pw, target]):
    
    models_score ={}
    for i in est:
        rskf = RepeatedStratifiedKFold(n_splits=splits, n_repeats=repeats, random_state=seed)
        results = cross_val_score(i[1], norm_df, np.array(column).reshape(-1,), cv=rskf, scoring=score)
        models_score.update({i[0] : results.mean()})
      
    results_dict[col_names[index]] = models_score

In [32]:
results_dict

{'Particle Spacing': {'LogisticRegression': 0.8927857327857328,
  'SVC': 0.9062493062493062,
  'GaussianNB': 0.8927857327857328,
  'BernoulliNB': 0.31610944610944613,
  'MultinomialNB': 0.858936618936619,
  'KNeighborsClassifier': 0.8927857327857328,
  'DecisionTreeClassifier': 0.7933244533244534,
  'GradientBoostingClassifier': 0.8450068450068451,
  'AdaBoostClassifier': 0.8662700262700265,
  'RandomForestClassifier': 0.8937862137862138,
  'ExtraTreesClassifier': 0.8794523994523995,
  'BaggingClassifier': 0.8667909867909868},
 'Particle Width': {'LogisticRegression': 0.6719607843137253,
  'SVC': 0.6739052287581697,
  'GaussianNB': 0.6355998373449353,
  'BernoulliNB': 0.6641830065359476,
  'MultinomialNB': 0.6719607843137253,
  'KNeighborsClassifier': 0.6680718954248365,
  'DecisionTreeClassifier': 0.6938482287501896,
  'GradientBoostingClassifier': 0.6411485790897555,
  'AdaBoostClassifier': 0.6252947760300701,
  'RandomForestClassifier': 0.72600175169783,
  'ExtraTreesClassifier': 0.

In [33]:
iterable = results_dict['Particle Spacing']
classifier = sorted(iterable, key=lambda i:iterable[i], reverse=True)[0]
print(classifier, iterable[classifier])

SVC 0.9062493062493062


In [34]:
iterable = results_dict['Particle Width']
classifier = sorted(iterable, key=lambda i:iterable[i], reverse=True)[0]
print(classifier, iterable[classifier])

RandomForestClassifier 0.72600175169783


In [35]:
iterable = results_dict['Cotton Crop']
classifier = sorted(iterable, key=lambda i:iterable[i], reverse=True)[0]
print(classifier, iterable[classifier])

SVC 0.9791230991230991


In [36]:
# "Particle Width" model
pw_model = DecisionTreeClassifier().fit(norm_df, np.array(pw).reshape(-1,))
# "Particle Spacing" model
ps_model = SVC().fit(norm_df, np.array(pw).reshape(-1,))
# "Cotton Crop" model
target_model = SVC().fit(norm_df, np.array(target).reshape(-1,))

In [37]:
gs = pd.DataFrame(enc.fit_transform(data['Grain Surface']))
gs.value_counts()

0
3    16
2    15
4    11
0     5
1     1
Name: count, dtype: int64

In [39]:
from sklearn.svm import LinearSVC
est =[]
est.append(('LogisticRegression', Pipeline([('LogisticRegression', LogisticRegression(multi_class='multinomial'))])))
est.append(('LinearSVC', Pipeline([('LinearSVC', LinearSVC())])))
est.append(('MultinomialNB', Pipeline([('MultinomialNB', MultinomialNB())])))
est.append(('DecisionTreeClassifier', Pipeline([('DecisionTreeClassifier', DecisionTreeClassifier(class_weight='balanced'))])))
est.append(('RandomForestClassifier', Pipeline([('RandomForestClassifier', RandomForestClassifier(class_weight='balanced'))])))
est.append(('ExtraTreesClassifier', Pipeline([('ExtraTreesClassifier', ExtraTreesClassifier(class_weight='balanced'))])))

In [40]:
import warnings
warnings.filterwarnings(action='ignore')

seed = 4
splits = 5
repeats = 3

score = 'f1_weighted'
models_score = {}

for i in est:
    rskf = RepeatedStratifiedKFold(n_splits=splits, n_repeats=repeats, random_state=seed)
    results = cross_val_score(i[1], norm_df, np.array(gs).reshape(-1,), cv=rskf, scoring=score)
    models_score.update({i[0] : results.mean()})
    
models_score

{'LogisticRegression': 0.5445802469135802,
 'LinearSVC': 0.5153668430335097,
 'MultinomialNB': 0.5229018759018759,
 'DecisionTreeClassifier': 0.5053544973544973,
 'RandomForestClassifier': 0.5631022927689594,
 'ExtraTreesClassifier': 0.6150440917107584}

In [41]:
classifier = sorted(models_score, key=lambda i:models_score[i], reverse=True)[0]
print(classifier, models_score[classifier])

ExtraTreesClassifier 0.6150440917107584


In [42]:
# "Grain Surface" model
gs_model = RandomForestClassifier().fit(norm_df, np.array(gs).reshape(-1,))

In [45]:
import tensorflow 
from tensorflow import keras
from tensorflow.keras import layers

tensorflow.config.run_functions_eagerly(True)

print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))


Num GPUs Available:  0


# Encoder

In [46]:
img_size_1 = 10
latent_space_dim = 2

In [47]:
# Encoder
x = tensorflow.keras.layers.Input(shape=(img_size_1, 1, 1), name="encoder_input")

encoder_conv_layer1 = tensorflow.keras.layers.Conv2D(filters=1, kernel_size=(3, 3), padding="same", strides=1, name="encoder_conv_1")(x)
encoder_norm_layer1 = tensorflow.keras.layers.BatchNormalization(name="encoder_norm_1")(encoder_conv_layer1)
encoder_activ_layer1 = tensorflow.keras.layers.LeakyReLU(name="encoder_leakyrelu_1")(encoder_norm_layer1)

encoder_conv_layer2 = tensorflow.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding="same", strides=1, name="encoder_conv_2")(encoder_activ_layer1)
encoder_norm_layer2 = tensorflow.keras.layers.BatchNormalization(name="encoder_norm_2")(encoder_conv_layer2)
encoder_activ_layer2 = tensorflow.keras.layers.LeakyReLU(name="encoder_activ_layer_2")(encoder_norm_layer2)

encoder_conv_layer3 = tensorflow.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", strides=2, name="encoder_conv_3")(encoder_activ_layer2)
encoder_norm_layer3 = tensorflow.keras.layers.BatchNormalization(name="encoder_norm_3")(encoder_conv_layer3)
encoder_activ_layer3 = tensorflow.keras.layers.LeakyReLU(name="encoder_activ_layer_3")(encoder_norm_layer3)

encoder_conv_layer4 = tensorflow.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", strides=2, name="encoder_conv_4")(encoder_activ_layer3)
encoder_norm_layer4 = tensorflow.keras.layers.BatchNormalization(name="encoder_norm_4")(encoder_conv_layer4)
encoder_activ_layer4 = tensorflow.keras.layers.LeakyReLU(name="encoder_activ_layer_4")(encoder_norm_layer4)

encoder_conv_layer5 = tensorflow.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", strides=1, name="encoder_conv_5")(encoder_activ_layer4)
encoder_norm_layer5 = tensorflow.keras.layers.BatchNormalization(name="encoder_norm_5")(encoder_conv_layer5)
encoder_activ_layer5 = tensorflow.keras.layers.LeakyReLU(name="encoder_activ_layer_5")(encoder_norm_layer5)

shape_before_flatten = tensorflow.keras.backend.int_shape(encoder_activ_layer5)[1:]
encoder_flatten = tensorflow.keras.layers.Flatten()(encoder_activ_layer5)

encoder_mu = tensorflow.keras.layers.Dense(units=latent_space_dim, name="encoder_mu")(encoder_flatten)
encoder_log_variance = tensorflow.keras.layers.Dense(units=latent_space_dim, name="encoder_log_variance")(encoder_flatten)

encoder_mu_log_variance_model = tensorflow.keras.models.Model(x, (encoder_mu, encoder_log_variance), name="encoder_mu_log_variance_model")

def sampling(mu_log_variance):
    mu, log_variance = mu_log_variance
    epsilon = tensorflow.keras.backend.random_normal(shape=tensorflow.keras.backend.shape(mu), mean=0.0, stddev=1.0)
    random_sample = mu + tensorflow.keras.backend.exp(log_variance/2) * epsilon
    return random_sample

encoder_output = tensorflow.keras.layers.Lambda(sampling, name="encoder_output")([encoder_mu, encoder_log_variance])

encoder = tensorflow.keras.models.Model(x, encoder_output, name="encoder_model")

encoder.summary()



Model: "encoder_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_input (InputLayer)  [(None, 10, 1, 1)]           0         []                            
                                                                                                  
 encoder_conv_1 (Conv2D)     (None, 10, 1, 1)             10        ['encoder_input[0][0]']       
                                                                                                  
 encoder_norm_1 (BatchNorma  (None, 10, 1, 1)             4         ['encoder_conv_1[0][0]']      
 lization)                                                                                        
                                                                                                  
 encoder_leakyrelu_1 (Leaky  (None, 10, 1, 1)             0         ['encoder_norm_1

# Decoder

In [48]:
decoder_input = tensorflow.keras.layers.Input(shape=(latent_space_dim), name="decoder_input")

decoder_dense_layer1 = tensorflow.keras.layers.Dense(units=640, name="decoder_dense_1")(decoder_input)
decoder_reshape = tensorflow.keras.layers.Reshape(target_shape=(10,1,64))(decoder_dense_layer1)

decoder_conv_tran_layer1 = tensorflow.keras.layers.Conv2DTranspose(filters=64, kernel_size=(3, 3), padding="same", strides=1, name="decoder_conv_tran_1")(decoder_reshape)
decoder_norm_layer1 = tensorflow.keras.layers.BatchNormalization(name="decoder_norm_1")(decoder_conv_tran_layer1)
decoder_activ_layer1 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_1")(decoder_norm_layer1)

decoder_conv_tran_layer2 = tensorflow.keras.layers.Conv2DTranspose(filters=64, kernel_size=(3, 3), padding="same", strides=1, name="decoder_conv_tran_2")(decoder_activ_layer1)
decoder_norm_layer2 = tensorflow.keras.layers.BatchNormalization(name="decoder_norm_2")(decoder_conv_tran_layer2)
decoder_activ_layer2 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_2")(decoder_norm_layer2)

decoder_conv_tran_layer3 = tensorflow.keras.layers.Conv2DTranspose(filters=64, kernel_size=(3, 3), padding="same", strides=1, name="decoder_conv_tran_3")(decoder_activ_layer2)
decoder_norm_layer3 = tensorflow.keras.layers.BatchNormalization(name="decoder_norm_3")(decoder_conv_tran_layer3)
decoder_activ_layer3 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_3")(decoder_norm_layer3)

decoder_conv_tran_layer4 = tensorflow.keras.layers.Conv2DTranspose(filters=1, kernel_size=(2, 2), padding="same", strides=1, name="decoder_conv_tran_4")(decoder_activ_layer3)
decoder_output = tensorflow.keras.layers.LeakyReLU(name="decoder_output")(decoder_conv_tran_layer4 )
decode = tensorflow.keras.layers.Reshape(target_shape=(10,1,1))
decoder = tensorflow.keras.models.Model(decoder_input, decoder_output, name="decoder_model")
 
decoder.summary()

Model: "decoder_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 2)]               0         
                                                                 
 decoder_dense_1 (Dense)     (None, 640)               1920      
                                                                 
 reshape (Reshape)           (None, 10, 1, 64)         0         
                                                                 
 decoder_conv_tran_1 (Conv2  (None, 10, 1, 64)         36928     
 DTranspose)                                                     
                                                                 
 decoder_norm_1 (BatchNorma  (None, 10, 1, 64)         256       
 lization)                                                       
                                                                 
 decoder_leakyrelu_1 (Leaky  (None, 10, 1, 64)       

# Loss Function (Reparametrization Trick)

In [49]:
def loss_func(encoder_mu, encoder_log_variance):
    def vae_reconstruction_loss(y_true, y_predict):
        
        reconstruction_loss_factor = 1000
        reconstruction_loss = tensorflow.keras.backend.mean(tensorflow.keras.backend.square(y_true-y_predict), axis=[1, 2, 3])
        return reconstruction_loss_factor * reconstruction_loss

    def vae_kl_loss(encoder_mu, encoder_log_variance):
        kl_loss = -0.5 * tensorflow.keras.backend.sum(1.0 + encoder_log_variance - tensorflow.keras.backend.square(encoder_mu) - tensorflow.keras.backend.exp(encoder_log_variance), axis=1)
        return kl_loss

    def vae_kl_loss_metric(y_true, y_predict):
        kl_loss = -0.5 * tensorflow.keras.backend.sum(1.0 + encoder_log_variance - tensorflow.keras.backend.square(encoder_mu) - tensorflow.keras.backend.exp(encoder_log_variance), axis=1)
        return kl_loss

    def vae_loss(y_true, y_predict):
        reconstruction_loss = vae_reconstruction_loss(y_true, y_predict)
        kl_loss = vae_kl_loss(y_true, y_predict)

        loss = reconstruction_loss + kl_loss
        return loss

    return vae_loss

# Combine Encoder and Decoder


In [50]:
vae_input = tensorflow.keras.layers.Input(shape=(img_size_1, 1, 1), name="VAE_input")
vae_encoder_output = encoder(vae_input)
vae_decoder_output = decoder(vae_encoder_output)
vae = tensorflow.keras.models.Model(vae_input, vae_decoder_output, name="VAE")
vae.summary()

Model: "VAE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 VAE_input (InputLayer)      [(None, 10, 1, 1)]        0         
                                                                 
 encoder_model (Functional)  (None, 2)                 94354     
                                                                 
 decoder_model (Functional)  (None, 10, 1, 1)          113729    
                                                                 
Total params: 208083 (812.82 KB)
Trainable params: 207249 (809.57 KB)
Non-trainable params: 834 (3.26 KB)
_________________________________________________________________


In [52]:
vae.compile(optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.0005), loss=loss_func(encoder_mu, encoder_log_variance))

In [53]:
norm_df.iloc[0].index.to_list()


['ph',
 'Temperature',
 'Humidity',
 'Density',
 'Electrical Conductivity',
 'N',
 'P',
 'K',
 'Ca',
 'Mg']

In [54]:
col_index  = {i:j for i, j in enumerate(norm_df.iloc[0].index.to_list())}
col_index

{0: 'ph',
 1: 'Temperature',
 2: 'Humidity',
 3: 'Density',
 4: 'Electrical Conductivity',
 5: 'N',
 6: 'P',
 7: 'K',
 8: 'Ca',
 9: 'Mg'}

# Reshaping norm_df before training


In [55]:
reshaped_array = np.array(norm_df).reshape(-1, 10, 1, 1)
reshaped_array.shape

(48, 10, 1, 1)

# Training

In [56]:
vae.fit(reshaped_array, reshaped_array, epochs=1000, batch_size=10, shuffle=True)


Epoch 1/1000



5/5 [==============================] - 1s 87ms/step - loss: 261.4782
Epoch 2/1000
5/5 [==============================] - 0s 85ms/step - loss: 169.0856
Epoch 3/1000
5/5 [==============================] - 0s 70ms/step - loss: 134.5856
Epoch 4/1000
5/5 [==============================] - 0s 70ms/step - loss: 106.3168
Epoch 5/1000
5/5 [==============================] - 0s 95ms/step - loss: 65.6247
Epoch 6/1000
5/5 [==============================] - 0s 68ms/step - loss: 56.3547
Epoch 7/1000
5/5 [==============================] - 0s 70ms/step - loss: 43.2072
Epoch 8/1000
5/5 [==============================] - 0s 70ms/step - loss: 43.0211
Epoch 9/1000
5/5 [==============================] - 0s 73ms/step - loss: 37.1851
Epoch 10/1000
5/5 [==============================] - 0s 77ms/step - loss: 31.0812
Epoch 11/1000
5/5 [==============================] - 0s 67ms/step - loss: 31.5761
Epoch 12/1000
5/5 [==============================] - 0s 74ms/step - loss: 31.4709
Epoch 13/1000
5/5 [===============

In [57]:
# Passing any random 2 numbers to decoder
z_sample = np.array([[-1, 0.6]])
decoder.predict(z_sample)

1/1 [==============================] - 0s 38ms/step


array([[[[0.6605407 ]],

        [[0.48373657]],

        [[0.65227956]],

        [[0.56107986]],

        [[0.56872344]],

        [[0.6922148 ]],

        [[0.70082   ]],

        [[0.30465388]],

        [[0.3683733 ]],

        [[0.30261832]]]], dtype=float32)

In [58]:
# Generating 4 rows of synthetic data

scale = 1.0
n = 2
grid_x = np.linspace(-scale, scale, n)
grid_y = np.linspace(-scale, scale, n)

In [59]:
synthetic_data = []
for i, yi in enumerate(grid_y):
    for j, xi in enumerate(grid_x):
        z_sample = np.array([[xi, yi]])
        x_decoded = decoder.predict(z_sample)
        synthetic_data.append(x_decoded[0])

1/1 [==============================] - 0s 14ms/step


In [60]:
np.array(synthetic_data).reshape(-1, 10)


array([[0.5927297 , 0.3806295 , 0.58217484, 0.4932755 , 0.5893052 ,
        0.6109066 , 0.6736249 , 0.30527037, 0.3541724 , 0.33924052],
       [0.68931806, 0.21709014, 0.51018775, 0.63217604, 0.6183236 ,
        0.51471657, 0.7225066 , 0.63689995, 0.40633804, 0.3512246 ],
       [0.6579671 , 0.48533925, 0.6546106 , 0.57057446, 0.54904604,
        0.72095907, 0.69122404, 0.29329753, 0.36768505, 0.30612957],
       [0.75126404, 0.2655719 , 0.7072726 , 0.5881994 , 0.4896699 ,
        0.7620142 , 0.6548728 , 0.4682477 , 0.54738003, 0.51247364]],
      dtype=float32)

In [61]:
pd.DataFrame(np.array(synthetic_data).reshape(-1, 10), columns=col_index.values())


,ph,Temperature,Humidity,Density,Electrical Conductivity,N,P,K,Ca,Mg
0,0.592730,0.380630,0.582175,0.493275,0.589305,0.610907,0.673625,0.305270,0.354172,0.339241
1,0.689318,0.217090,0.510188,0.632176,0.618324,0.514717,0.722507,0.636900,0.406338,0.351225
2,0.657967,0.485339,0.654611,0.570574,0.549046,0.720959,0.691224,0.293298,0.367685,0.306130
3,0.751264,0.265572,0.707273,0.588199,0.489670,0.762014,0.654873,0.468248,0.547380,0.512474


In [62]:
%%time

scale = 1.0
n = 100
grid_x = np.linspace(-scale, scale, n)
grid_y = np.linspace(-scale, scale, n)

synthetic_data = []
for i, yi in enumerate(grid_y):
    for j, xi in enumerate(grid_x):
        z_sample = np.array([[xi, yi]])
        x_decoded = decoder.predict(z_sample)
        synthetic_data.append(x_decoded[0])
        
syn_data = np.array(synthetic_data).reshape(-1, 10)
synth_data = pd.DataFrame(syn_data, columns=col_index.values())

1/1 [==============================] - 0s 42ms/step
CPU times: total: 17min 39s
Wall time: 16min 12s


In [63]:
synth_data

,ph,Temperature,Humidity,Density,Electrical Conductivity,N,P,K,Ca,Mg
0,0.592730,0.380630,0.582175,0.493275,0.589305,0.610907,0.673625,0.305270,0.354172,0.339241
1,0.595020,0.380018,0.582570,0.493800,0.591148,0.612374,0.674034,0.309545,0.356519,0.341657
2,0.597252,0.379275,0.582895,0.494429,0.592945,0.613846,0.674399,0.313799,0.358872,0.344054
3,0.599789,0.378791,0.583180,0.495068,0.594765,0.615340,0.674764,0.317743,0.361340,0.346267
4,0.602320,0.378435,0.583416,0.495682,0.596603,0.616757,0.675157,0.321356,0.363997,0.348404
...,...,...,...,...,...,...,...,...,...,...
9995,0.755881,0.275221,0.710292,0.589107,0.490823,0.768501,0.660687,0.465794,0.542940,0.511202
9996,0.754739,0.272830,0.709635,0.588942,0.490384,0.767073,0.659433,0.466175,0.543836,0.511633
9997,0.753584,0.270422,0.708893,0.588716,0.489979,0.765583,0.658015,0.466769,0.544916,0.511930
9998,0.752424,0.267997,0.708098,0.588442,0.489834,0.763855,0.656433,0.467549,0.546136,0.512224


# Classifying the new synthetic data into previous categorical columns

In [64]:
# Particle Width
pw_predicted = pw_model.predict(synth_data)
# Particle Spacing
ps_predicted = ps_model.predict(synth_data)
# Cotton Crop
target_predicted = target_model.predict(synth_data)
# Grain Surface
gs_predicted = gs_model.predict(synth_data)

In [65]:
pw_d = pd.DataFrame(pw_predicted, columns=['Particle Width'])
ps_d = pd.DataFrame(ps_predicted, columns=['Particle Spacing'])
cc_d = pd.DataFrame(target_predicted, columns=['Cotton Crop'])
gs_d = pd.DataFrame(gs_predicted, columns=['Grain Surface'])

cat_col = pd.concat([pw_d, ps_d, cc_d, gs_d], axis=1)
cat_col

,Particle Width,Particle Spacing,Cotton Crop,Grain Surface
0,0,0,1,2
1,0,0,1,2
2,0,0,1,2
3,0,0,1,2
4,0,0,1,2
...,...,...,...,...
9995,0,0,1,2
9996,0,0,1,2
9997,0,0,1,2
9998,0,0,1,2


In [66]:
pw_d.value_counts()


Particle Width
0                 10000
Name: count, dtype: int64

In [67]:
ps_d.value_counts()


Particle Spacing
0                   10000
Name: count, dtype: int64

In [68]:
cc_d.value_counts()


Cotton Crop
1              10000
Name: count, dtype: int64

In [69]:
gs_d.value_counts()


Grain Surface
2                10000
Name: count, dtype: int64

In [70]:
new_synth_data = pd.concat([synth_data, cat_col], axis=1)
new_synth_data

,ph,Temperature,Humidity,Density,Electrical Conductivity,N,P,K,Ca,Mg,Particle Width,Particle Spacing,Cotton Crop,Grain Surface
0,0.592730,0.380630,0.582175,0.493275,0.589305,0.610907,0.673625,0.305270,0.354172,0.339241,0,0,1,2
1,0.595020,0.380018,0.582570,0.493800,0.591148,0.612374,0.674034,0.309545,0.356519,0.341657,0,0,1,2
2,0.597252,0.379275,0.582895,0.494429,0.592945,0.613846,0.674399,0.313799,0.358872,0.344054,0,0,1,2
3,0.599789,0.378791,0.583180,0.495068,0.594765,0.615340,0.674764,0.317743,0.361340,0.346267,0,0,1,2
4,0.602320,0.378435,0.583416,0.495682,0.596603,0.616757,0.675157,0.321356,0.363997,0.348404,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.755881,0.275221,0.710292,0.589107,0.490823,0.768501,0.660687,0.465794,0.542940,0.511202,0,0,1,2
9996,0.754739,0.272830,0.709635,0.588942,0.490384,0.767073,0.659433,0.466175,0.543836,0.511633,0,0,1,2
9997,0.753584,0.270422,0.708893,0.588716,0.489979,0.765583,0.658015,0.466769,0.544916,0.511930,0,0,1,2
9998,0.752424,0.267997,0.708098,0.588442,0.489834,0.763855,0.656433,0.467549,0.546136,0.512224,0,0,1,2


In [71]:
new_synth_data.to_csv('synthetic_scaled_data.csv')
